In [ ]:
# Lista el contenido del directorio raíz en DBFS (cluster distribuido)
root_files = dbutils.fs.ls('/')
for file in root_files:
    print(file.path)

dbfs:/FileStore/
dbfs:/databricks-datasets/
dbfs:/databricks-results/
dbfs:/tmp/


In [ ]:
# Lista el contenido del directorio /FileStore, donde se cargó el json con las credenciales de la API de Kaggle
filestore_files = dbutils.fs.ls("/FileStore/shared_uploads/YOUR_DATABRICKS_USER")
for file in filestore_files:
    print(file.path)

dbfs:/FileStore/shared_uploads/cristian.neumann@usach.cl/kaggle.json


In [ ]:
# Leer el contenido de un archivo
dbutils.fs.head("dbfs:/FileStore/shared_uploads/YOUR_DATABRICKS_USER/kaggle.json")

Out[11]: '{"username":"cristianneumann","key":"51956eb6263a97a83b2c65c47447a264"}'

In [ ]:
# Instalar librerias de Kaggle
%pip install kaggle

In [ ]:
# Configurar credenciales de Kaggle
import os
import json

# Cargar el archivo kaggle.json que has subido a DBFS
dbutils.fs.cp("dbfs:/FileStore/shared_uploads/YOUR_DATABRICKS_USER/kaggle.json", "file:/root/.kaggle/kaggle.json")

# Establecer los permisos adecuados
os.chmod("/root/.kaggle/kaggle.json", 0o600)

# Verificar que las credenciales estén configuradas correctamente
with open("/root/.kaggle/kaggle.json", "r") as f:
    kaggle_credentials = json.load(f)
    print(kaggle_credentials)

{'username': 'cristianneumann', 'key': '51956eb6263a97a83b2c65c47447a264'}


In [ ]:
# Listar los datasets de kaggle disponibles en su API
import kaggle

# Funcion que transforma el contenido de dataset.size (string) en número
def size_to_bytes(size_str):
    size_str = size_str.strip().upper()
    size_units = {"KB": 1024, "MB": 1024**2, "GB": 1024**3, "TB": 1024**4}
    if size_str[-2:] in size_units:
        return float(size_str[:-2]) * size_units[size_str[-2:]]
    elif size_str[-1] == "B":
        return float(size_str[:-1])
    else:
        return float(size_str)

# Buscar datasets grandes. Probamos con la etiqueta "parquet", pudiendo existir muchas otras.
datasets = kaggle.api.dataset_list(search="parquet")

# Filtrar datasets por tamaño (mostrando aquellos mayores a 0,5 GB)
large_datasets = [dataset for dataset in datasets if size_to_bytes(dataset.size) > (1024**3)/2]

# Mostrar los datasets grandes
for dataset in large_datasets:
    print(dataset, dataset.size)

ruchi798/parquet-files-amexdefault-prediction 22GB
odins0n/amex-parquet 9GB
robikscube/ubiquant-parquet 13GB
raddar/amex-data-integer-dtypes-parquet-format 4GB
rohanrao/riiid-train-data-multiple-formats 4GB
robikscube/ai4code-parquet-tabular 785MB
ravi20076/optiver-memoryreduceddatasets 2GB
jjinho/wikipedia-20230701 12GB
alvinleenh/tps-rocket-league-data-float16-parquet-format 2GB
columbia2131/otto-chunk-data-inparquet-format 2GB
ryati131457/riiid-parquet-files 564MB
samuelcortinhas/tps-nov-22-dataset-in-parquet-format 1GB
mustafakeser4/tpsoct22-parquet 3GB
cdeotte/otto-validation 1GB
titericz/leap-dataset-giba 16GB
pedrocouto39/jane-street-market-train-data-best-formats 10GB


In [ ]:
# Buscar el dataset "amex-data-integer-dtypes-parquet-format" del usuario "raddar"
dataset = kaggle.api.dataset_list(search="raddar/amex-data-integer-dtypes-parquet-format")
print(dataset, dataset[0].size)

[raddar/amex-data-integer-dtypes-parquet-format] 4GB


In [ ]:
# Importar el dataset "amex-data-integer-dtypes-parquet-format" en ruta del nodo local /tmp/wiki_data/. Esto, pues la API de Kaggle descargará el archivo al nodo local.
import kaggle
kaggle.api.dataset_download_files('raddar/amex-data-integer-dtypes-parquet-format', path='/tmp/wiki_data/', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/raddar/amex-data-integer-dtypes-parquet-format


In [ ]:
# Lista el contenido del directorio Local: file:/tmp/wiki_data/
root_local_files = dbutils.fs.ls('file:/tmp/wiki_data/')
for file in root_local_files:
    print(file.path)

In [ ]:
# Mover el dataset descargado desde el sistema de archivos local al distribuido (DBFS), para que persista
dbutils.fs.cp("file:/tmp/wiki_data/", "dbfs:/tmp/wiki_data/", recurse=True)

Out[13]: True

In [ ]:
# Volvemos a listar el contenido del directorio raíz en DBFS para verificar que aparezca el path del dataset copiado
root_files = dbutils.fs.ls('/')
for file in root_files:
    print(file.path)

dbfs:/FileStore/
dbfs:/databricks-datasets/
dbfs:/databricks-results/
dbfs:/tmp/
